In [41]:
# Importing all the required modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

# Libraries related to outlier detection
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# Encoding and Standardization
from sklearn.preprocessing import LabelEncoder, RobustScaler

In [42]:
# Ignore all warnings

import warnings
warnings.filterwarnings("ignore")

In [43]:
# Load the CSV and Excel files into dataframes
file_2015 = "15tstcar.csv"
file_2016 = "16tstcar.csv"
file_2017 = "17tstcar-2018-05-30.xlsx"
file_2018 = "18tstcar-2018-10-24.xlsx"
file_2019 = "19tstcar-2020-10-02.xlsx"
file_2020 = "20tstcar-2021-03-02.xlsx"
file_2021 = "21-tstcar-2022-04-15.xlsx"
file_2022 = "22-testcar-2023-06-13.xlsx"
file_2023 = "23-testcar-2024-05-17_0.xlsx"
file_2024 = "24-testcar-2024-05-17_0 (1).xlsx"

In [44]:
# Load CSV files
df_2015 = pd.read_csv(file_2015)
df_2016 = pd.read_csv(file_2016)

# Load Excel files
df_2017 = pd.read_excel(file_2017)
df_2018 = pd.read_excel(file_2018)
df_2019 = pd.read_excel(file_2019)
df_2020 = pd.read_excel(file_2020)
df_2021 = pd.read_excel(file_2021)
df_2022 = pd.read_excel(file_2022)
df_2023 = pd.read_excel(file_2023)
df_2024 = pd.read_excel(file_2024)

In [45]:
df = pd.concat([df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023, df_2024], ignore_index=True)
df.shape

(45479, 67)

### Step 1: Data preprocessing and Exploration

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45479 entries, 0 to 45478
Data columns (total 67 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Model Year                      45479 non-null  int64  
 1   Vehicle Manufacturer Name       45479 non-null  object 
 2   Veh Mfr Code                    45479 non-null  object 
 3   Represented Test Veh Make       45479 non-null  object 
 4   Represented Test Veh Model      45479 non-null  object 
 5   Test Vehicle ID                 45479 non-null  object 
 6   Test Veh Configuration #        45479 non-null  int64  
 7   Test Veh Displacement (L)       45479 non-null  float64
 8   Actual Tested Testgroup         45479 non-null  object 
 9   Vehicle Type                    45479 non-null  object 
 10  Rated Horsepower                45479 non-null  int64  
 11  # of Cylinders and Rotors       43103 non-null  float64
 12  Engine Code                     

Check for missing values

In [47]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    total_missing = missing_values.sum()
    total_rows = len(df)

    if total_missing > 0:
        print("Missing values found in the following columns (sorted by missing count):")
        # Calculate the percentage of missing values
        missing_percentage = round(((missing_values / total_rows) * 100), 2)
        # Combine missing counts and percentages
        data_types = df.dtypes
        missing_info = pd.DataFrame({'Missing Values': missing_values, 'Percentage (%)': missing_percentage, 'Data Type': data_types})
        # Filter and sort by missing values
        sorted_missing_info = missing_info[missing_info['Missing Values'] > 0].sort_values(by='Missing Values', ascending=False)
        #print(sorted_missing_info)
        return sorted_missing_info
    else:
        print("No missing values found in the dataframe.")


In [48]:
check_missing_values(df)

Missing values found in the following columns (sorted by missing count):


,Missing Values,Percentage (%),Data Type
FE Bag 4,43715,96.12,float64
Averaging Weighting Factor,43686,96.06,float64
Averaging Group ID,43686,96.06,object
ADFE Test Number,40746,89.59,object
ADFE Total Road Load HP,40746,89.59,float64
ADFE N/V Ratio,40746,89.59,float64
ADFE Equiv. Test Weight (lbs.),40742,89.58,float64
PM (g/mi),38232,84.07,float64
FE Bag 3,27793,61.11,float64
FE Bag 1,24410,53.67,float64


There are no discrepancies with regard to data types of the various columns across the datasets. However, there are several attributes with significant number of null values in the dataset. Depending on the importance of the variable, we either perform imputation or remove the attribute from the list of predictor variables.

From the above list, we remove the attributes with greater than 50% of the values missing (ADFE Test Number, ADFE Total Road Load HP, ADFE N/V Ratio, ADFE Equiv.Test Weight, PM(g/mi), FE Bag 3, FE Bag 1, FE Bag 2 and N2O (g/mi)). 

In [49]:
def drop_columns_with_missing_values(df, columns_to_drop):
    df_reduced = df.copy()
    
    # Drop the specified columns
    df_reduced = df_reduced.drop(columns=columns_to_drop, errors='ignore')
    
    return df_reduced

In [50]:
columns_to_remove = [
    'FE Bag 4', 'Averaging Weighting Factor', 'Averaging Group ID', 
    'ADFE Test Number', 'ADFE Total Road Load HP', 'ADFE N/V Ratio', 
    'ADFE Equiv. Test Weight (lbs.)', 'PM (g/mi)', 'FE Bag 3', 
    'FE Bag 1', 'FE Bag 2', 'N2O (g/mi)'
]
df_reduced = drop_columns_with_missing_values(df, columns_to_remove)

In [51]:
check_missing_values(df_reduced)

Missing values found in the following columns (sorted by missing count):


,Missing Values,Percentage (%),Data Type
DT-Inertia Work Ratio Rating,9389,20.64,float64
DT-Absolute Speed Change Ratg,9389,20.64,float64
DT-Energy Economy Rating,9389,20.64,float64
CH4 (g/mi),8816,19.38,float64
NOx (g/mi),6266,13.78,float64
THC (g/mi),6008,13.21,float64
CO (g/mi),5902,12.98,float64
CO2 (g/mi),2913,6.41,float64
Aftertreatment Device Cd,2444,5.37,object
Aftertreatment Device Desc,2444,5.37,object


In [53]:
## New Mode Imputation

from sklearn.experimental import enable_iterative_imputer  # Required for IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer, IterativeImputer
import numpy as np
import pandas as pd

def impute_data(df, knn_neighbors=7):
    # Create a copy of the DataFrame to avoid changing the original
    df_imputed = df.copy()
    
    # Separate numeric and categorical columns
    numeric_cols = df_imputed.select_dtypes(include=['number']).columns
    categorical_cols = [
        'Vehicle Manufacturer Name', 
        'Vehicle Type', 
        'Tested Transmission Type Code', 
        'Transmission Lockup?', 
        'Drive System Code', 
        'Transmission Overdrive Desc', 
        'Shift Indicator Light Use Desc', 
        'Test Procedure Description', 
        'Test Fuel Type Description', 
        'Test Category', 
        'Aftertreatment Device Cd',
        'Aftertreatment Device Desc',  # Explicitly add missing categorical cols
        'Engine Code'
    ]

    # 1. KNN Imputation for numeric columns
    knn_imputer = KNNImputer(n_neighbors=knn_neighbors)
    df_imputed[numeric_cols] = pd.DataFrame(knn_imputer.fit_transform(df_imputed[numeric_cols]), 
                                            columns=numeric_cols, index=df_imputed.index)

    # 2. Encode categorical columns as integers and replace missing values with np.nan
    categorical_codes = {}
    for col in categorical_cols:
        df_imputed[col] = pd.Categorical(df_imputed[col]).codes
        df_imputed[col].replace(-1, np.nan, inplace=True)  # Replace -1 with np.nan for missing values
        categorical_codes[col] = pd.Categorical(df[col].dropna()).categories  # Save original categories

    # Apply MICE imputer to encoded categorical columns
    mice_imputer = IterativeImputer(max_iter=10, random_state=42, sample_posterior=True)
    try:
        imputed_categorical = pd.DataFrame(mice_imputer.fit_transform(df_imputed[categorical_cols]), 
                                           columns=categorical_cols, index=df_imputed.index)
    except Exception as e:
        print("MICE imputation failed for categorical columns:", e)
        # Fallback: Simple mode imputation
        simple_imputer = SimpleImputer(strategy='most_frequent')
        imputed_categorical = pd.DataFrame(simple_imputer.fit_transform(df_imputed[categorical_cols]), 
                                           columns=categorical_cols, index=df_imputed.index)

    # Round imputed values to nearest integer, clip to valid range, and reconvert to categories
    for col in categorical_cols:
        categories = categorical_codes[col]
        imputed_categorical[col] = imputed_categorical[col].round().clip(0, len(categories) - 1).astype(int)
        df_imputed[col] = pd.Categorical.from_codes(imputed_categorical[col], categories=categories)

    return df_imputed


In [54]:
df_imputed = impute_data(df_reduced)

In [55]:
check_missing_values(df_imputed)

No missing values found in the dataframe.


In [56]:
categorical_cols = df_reduced.select_dtypes(include=['object']).columns
for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"Column '{col}' has {unique_count} unique values.")

Column 'Vehicle Manufacturer Name' has 55 unique values.
Column 'Veh Mfr Code' has 54 unique values.
Column 'Represented Test Veh Make' has 97 unique values.
Column 'Represented Test Veh Model' has 2326 unique values.
Column 'Test Vehicle ID' has 4366 unique values.
Column 'Actual Tested Testgroup' has 2432 unique values.
Column 'Vehicle Type' has 3 unique values.
Column 'Engine Code' has 2031 unique values.
Column 'Tested Transmission Type Code' has 8 unique values.
Column 'Tested Transmission Type' has 8 unique values.
Column 'Transmission Lockup?' has 2 unique values.
Column 'Drive System Code' has 5 unique values.
Column 'Drive System Description' has 5 unique values.
Column 'Transmission Overdrive Desc' has 3 unique values.
Column 'Shift Indicator Light Use Desc' has 4 unique values.
Column 'Test Number' has 23168 unique values.
Column 'Test Originator' has 2 unique values.
Column 'Analytically Derived FE?' has 2 unique values.
Column 'Test Procedure Description' has 16 unique val

Among the categorical variables, some of them have extremely large number of values such as Represented Test Veh Model, Test Vehicle ID, Actual Tested Testgroup and Engine Code. Choosing these values will create 

When selecting categorical variables, it is important to consider its relevance to the response variable as well as the number of unique values. Some of the categorical attributes such as Represented Test Veh Model, Test Vehicle ID, Actual Tested Testgroup and Engine Code have an extremely large number of values. Encoding these values will lead to issues with model interpretability and complexity. Taking these factors into account we choose the following categorical variables - **Vehicle Manufacturer Name, Vehicle Type, TTested Transmission Type Code, Transmission Lockup?, Drive System Code, Transmission Overdrive Desc, Shift Indicator Light Use Desc, Test Procedure Description, Test Fuel Type Description, Test Category and Aftertreatment Device Cd**

In [57]:
df_imputed.describe()

,Model Year,Test Veh Configuration #,Test Veh Displacement (L),Rated Horsepower,# of Cylinders and Rotors,# of Gears,Transmission Overdrive Code,Equivalent Test Weight (lbs.),Axle Ratio,N/V Ratio,...,RND_ADJ_FE,DT-Inertia Work Ratio Rating,DT-Absolute Speed Change Ratg,DT-Energy Economy Rating,Target Coef A (lbf),Target Coef B (lbf/mph),Target Coef C (lbf/mph**2),Set Coef A (lbf),Set Coef B (lbf/mph),Set Coef C (lbf/mph**2)
count,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000,...,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000,45479.000000
mean,2019.392555,1.176169,3.390352,298.049188,5.443652,6.281119,1.951538,4396.410981,3.616840,32.950513,...,49.982547,8.002946,7.895986,7.448573,39.257251,0.231317,0.023138,11.091428,0.111210,0.023945
std,2.865135,2.028842,7.182849,151.481306,1.816387,2.717309,0.214743,858.872627,1.154017,46.049127,...,363.583723,24.669669,24.605541,24.850952,11.942311,0.322410,0.007210,11.292163,0.298469,0.094805
min,2015.000000,0.000000,0.001000,1.000000,2.000000,1.000000,1.000000,2125.000000,0.000000,0.000000,...,0.000000,-29.170000,-24.010000,-17.818000,9.870000,-0.847300,0.002480,-99.900000,-1.975200,-0.090640
25%,2017.000000,0.000000,2.000000,183.000000,4.000000,6.000000,2.000000,3750.000000,3.140000,24.300000,...,24.000000,-0.620000,-0.507203,-0.568000,30.620000,0.054260,0.018111,5.260000,-0.020000,0.017580
50%,2019.000000,0.000000,2.500000,275.000000,4.857143,7.000000,2.000000,4250.000000,3.420000,26.900000,...,31.100000,0.620979,0.410000,-0.095857,38.100000,0.251600,0.021903,11.400000,0.107300,0.020940
75%,2022.000000,1.000000,3.500000,362.000000,6.000000,8.000000,2.000000,5000.000000,3.770000,31.200000,...,41.100000,2.704431,1.990105,0.389000,46.700000,0.422400,0.026420,17.788500,0.235580,0.026110
max,2024.000000,25.000000,99.999000,1839.000000,16.000000,10.000000,2.000000,9000.000000,9.990000,999.900000,...,10000.000000,99.990000,99.990000,99.990000,150.000000,2.408160,0.218550,150.200000,9.999990,9.999999


### Performing encoding of selected categorical variables

Encoding categorical attributes with high cardinality values can increase the complexity and lead to model interpretability issues. For example, Vehicle Manufacturer Name attribute has 55 unique values. So, performing one-hot encoding will result in the addition of 55 new columns to the dataset. In order to prevent that, we will first re-categorize the various vehicle manufacturer names based on manufacturer groups and then perform one-hot encoding. 

In [58]:
brand_groups = {
    'Volkswagen Group': ['Audi', 'Bentley', 'Bugatti', 'Volkswagen', 'Porsche', 'Lamborghini'],
    'FCA Group': ['FCA Italy', 'FCA US LLC', 'Ferrari', 'Maserati'],
    'Ford Motor Company': ['FOMOCO'],
    'General Motors': ['GM'],
    'Hyundai Motor Group': ['Hyundai', 'Kia', 'Genesis'],
    'BMW Group': ['BMW', 'Mini', 'Rolls-Royce'],
    'Toyota Group': ['Toyota', 'Lexus'],
    'Daimler AG': ['Mercedes-Benz'],
    'Honda': ['Honda', 'Acura'],
    'Tata Motors': ['Jaguar Land Rover L'],
    'Electric Vehicle Manufacturers': ['Tesla', 'Rivian Automotive L', 'Lucid USA, Inc', 'Fisker Group Inc.'],
    'High-Performance and Specialty': [
        'Aston Martin', 'McLaren Automotive', 'Pagani Automobili S', 'Koenigsegg', 
        'Rimac Automobili', 'Maserati', 'Bugatti Rimac LLC', 'RUF'
    ],
    'Asian Manufacturers': ['BYD Motors Inc.', 'Nissan', 'Mazda', 'Mitsubishi Motors Co', 'Subaru'],
    'Other': [
        'Quantum Fuel System', 'CSC', 'Mobility Ventures L', 'Bluecar, SAS', 
        'Ineos Automotive Li', 'Lordstown EV Corpor', 'Canoo Technologies', 
        'Faraday&Future Inc.', 'Vinfast Trading and', 'Mullen Automotive'
    ]
}

def map_manufacturer_group(df, column, brand_groups):
    # Create a reverse mapping dictionary from brand_groups
    group_mapping = {manufacturer: group for group, manufacturers in brand_groups.items() for manufacturer in manufacturers}
    
    # Map the column values to their respective groups
    df[column] = df[column].map(group_mapping) 
    
    return df

In [59]:
# Usage
df_vmn = map_manufacturer_group(df_imputed, 'Vehicle Manufacturer Name', brand_groups)

In [60]:
def label_encode_df(df):
    # Create a copy of the DataFrame to avoid modifying the original
    df_encoded = df.copy()
    
    # Loop through all categorical columns and apply label encoding
    for col in df_encoded.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))  # Convert to string in case of mixed types
    
    return df_encoded

In [61]:
df_encoded = label_encode_df(df_vmn)

In [62]:
len(df_encoded.columns)

55

### Removing multivariate outliers using a combination of three techniques (KNN, Decision Tree and Mahalanobis distance)

In [63]:
def removing_outliers(df, selected_columns):
    # Keep the other columns intact
    other_columns = df.drop(columns=selected_columns)
    
    # Select only the specified columns for outlier detection
    df_num = df[selected_columns]
    
    # Row count before removing outliers
    before_count = len(df_num)
    
    # Local Outlier Factor with lower contamination (1-2%)
    lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)
    y_pred_lof = lof.fit_predict(df_num)
    
    # Isolation Forest with lower contamination (1-2%)
    iforest = IsolationForest(n_estimators=100, contamination=0.01)
    y_pred_if = iforest.fit_predict(df_num)
    
    # Elliptic Envelope with lower contamination (1-2%)
    rob_cov = EllipticEnvelope(contamination=0.01)
    rob_cov.fit(df_num)
    y_pred_rob = rob_cov.predict(df_num)
    
    # Assign outlier labels
    df_num["y_pred_lof"] = y_pred_lof
    df_num["y_pred_if"] = y_pred_if
    df_num["y_pred_rob"] = y_pred_rob
    
    df_num["y_pred_lof_2"] = np.where(df_num["y_pred_lof"] < 0, -1, 0)
    df_num["y_pred_if_2"] = np.where(df_num["y_pred_if"] < 0, -1, 0)
    df_num["y_pred_rob_2"] = np.where(df_num["y_pred_rob"] < 0, -1, 0)
    
    # Crosstab for checking consistency (optional for debugging)
    pd.crosstab(df_num["y_pred_if_2"], df_num["y_pred_rob_2"])
    
    # Sum of outlier labels from all methods
    df_num["all_out"] = df_num.loc[:, ["y_pred_if_2", "y_pred_rob_2", "y_pred_lof_2"]].sum(axis=1)
    
    # Filter out outliers
    df_num_filtered = df_num[df_num["all_out"] > -1]
    
    # Row count after removing outliers
    after_count = len(df_num_filtered)
    
    # Calculate and print the percentage of rows removed
    percentage_removed = ((before_count - after_count) / before_count) * 100
    print(f"Percentage of rows removed as outliers: {percentage_removed:.2f}%")
    
    # Drop outlier-related columns used for filtering
    df_num_filtered = df_num_filtered.drop(columns=["y_pred_lof", "y_pred_if", "y_pred_rob", "y_pred_lof_2", "y_pred_if_2", "y_pred_rob_2", "all_out"])
    
    # Concatenate the filtered numerical columns with the other non-imputed columns
    final_df = pd.concat([other_columns.reset_index(drop=True), df_num_filtered.reset_index(drop=True)], axis=1)
    
    return final_df

In [64]:
df_outliers_removed = removing_outliers(df_encoded, df_imputed.select_dtypes(include=['object']).columns.tolist())

Percentage of rows removed as outliers: 2.93%


In [65]:
df_outliers_removed.columns

Index(['Model Year', 'Test Veh Configuration #', 'Test Veh Displacement (L)',
       'Vehicle Type', 'Rated Horsepower', '# of Cylinders and Rotors',
       'Engine Code', 'Tested Transmission Type Code', '# of Gears',
       'Transmission Lockup?', 'Drive System Code',
       'Transmission Overdrive Code', 'Transmission Overdrive Desc',
       'Equivalent Test Weight (lbs.)', 'Axle Ratio', 'N/V Ratio',
       'Shift Indicator Light Use Cd', 'Shift Indicator Light Use Desc',
       'Test Procedure Cd', 'Test Procedure Description', 'Test Fuel Type Cd',
       'Test Fuel Type Description', 'Test Category', 'THC (g/mi)',
       'CO (g/mi)', 'CO2 (g/mi)', 'NOx (g/mi)', 'CH4 (g/mi)', 'RND_ADJ_FE',
       'DT-Inertia Work Ratio Rating', 'DT-Absolute Speed Change Ratg',
       'DT-Energy Economy Rating', 'Target Coef A (lbf)',
       'Target Coef B (lbf/mph)', 'Target Coef C (lbf/mph**2)',
       'Set Coef A (lbf)', 'Set Coef B (lbf/mph)', 'Set Coef C (lbf/mph**2)',
       'Aftertreatment De

### Performing standardization 

In [68]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

def standardize_numerical_columns(df):
    # Select only numerical columns
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    
    # Create a copy of the DataFrame to avoid modifying the original
    df_standardized = df.copy()
    
    # Initialize the StandardScaler
    scaler = RobustScaler()
    
    # Fit and transform only the numerical columns
    df_standardized[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    
    return df_standardized

# Usage
df_standardized = standardize_numerical_columns(df_outliers_removed)

We select several vehicle specific characteristics, emission specific factors along with factors related to drive and transmission for predicting CO emission and attributes related to Engine and Powertrain, Aerodynamics and Resistance, Vehicle load and Weight for  Miles per Gallon prediction.

In [69]:
# Attributes for predicting miles per gallon (RND_ADJ_FE)
mpg_attributes = [
    'Vehicle Manufacturer Name', 'Vehicle Type', 'Rated Horsepower', '# of Cylinders and Rotors',
    'Test Veh Displacement (L)', 'Transmission Overdrive Code', '# of Gears', 'Axle Ratio',
    'Target Coef A (lbf)', 'Target Coef B (lbf/mph)', 'Target Coef C (lbf/mph**2)',
    'Set Coef A (lbf)', 'Set Coef B (lbf/mph)', 'Set Coef C (lbf/mph**2)',
    'Equivalent Test Weight (lbs.)', 'DT-Inertia Work Ratio Rating', 'RND_ADJ_FE'
]

# Attributes for predicting CO emissions (CO (g/mi))
co_emission_attributes = [
    'Vehicle Manufacturer Name', 'Vehicle Type', 'Rated Horsepower', '# of Cylinders and Rotors',
    'Test Veh Displacement (L)', 'Axle Ratio', 'N/V Ratio', 'THC (g/mi)', 'NOx (g/mi)',
    'CH4 (g/mi)', 'Aftertreatment Device Desc', 'Test Fuel Type Description',
    'Drive System Description', 'Transmission Overdrive Desc', 'Equivalent Test Weight (lbs.)',
    'Shift Indicator Light Use Desc', 'CO (g/mi)'
]